Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in investment options input csv
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# eliminate repeat symbols
symbols = set(etfs)

In [7]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..........................................................................


In [8]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [9]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['SPY']

18.665073401373

In [10]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [11]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['SPY']

0.16033580120221275

In [12]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.10642810860176354

In [13]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['SPY'])

Date
2015-01-31   177.43
2015-02-28   187.40
2015-03-31   184.46
2015-04-30   186.27
2015-05-31   188.66
2015-06-30   184.83
2015-07-31   189.01
2015-08-31   177.49
2015-09-30   172.96
2015-10-31   187.67
2015-11-30   188.36
2015-12-31   185.10
2016-01-31   175.89
2016-02-29   175.74
2016-03-31   187.56
2016-04-30   188.30
2016-05-31   191.50
2016-06-30   192.17
2016-07-31   199.18
2016-08-31   199.42
2016-09-30   199.43
2016-10-31   195.97
2016-11-30   203.19
2016-12-31   207.31
2017-01-31   211.02
2017-02-28   219.31
2017-03-31   219.58
2017-04-30   221.76
2017-05-31   224.89
2017-06-30   226.33
2017-07-31   230.98
2017-08-31   231.65
2017-09-30   236.32
2017-10-31   241.89
2017-11-30   249.28
2017-12-31   252.31
2018-01-31   266.53
2018-02-28   256.84
2018-03-31   249.80
2018-04-30   251.09
2018-05-31   257.19
2018-06-30   258.67
2018-07-31   268.25
2018-08-31   276.81
2018-09-30   278.46
2018-10-31   259.22
2018-11-30   264.03
2018-12-31   240.78
2019-01-31   260.06
2019-02-28   26

In [14]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['SPY']

Date
2015-01-31     nan
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2020-10-31   -0.02
2020-11-30    0.11
2020-12-31    0.04
2021-01-31   -0.01
2021-02-28    0.06
Freq: M, Name: SPY, Length: 74, dtype: float64

In [15]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['SPY']

0.18278253245752824

In [16]:
# read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# for each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = []
for line in lines:
    # copy empty and comment lines
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # split line by comma delimiter; strip the fields
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # copy header
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')